In [1]:
import os
import glob
import pandas as pd
import numpy as np

In [2]:
#parameters to clean up data

visit_length = 150 # anything less than 200 frames gets dropped
m = 0.4 # mean value of Prob(proboscis)

frames_btw_visits = 600

In [3]:
def MergeVisits(df, frames):
    idx = df[df['frames_inbetween'] <= frames].index.values
    if len(idx)>1:
        conseq = idx[:-1] - idx[1:]
        for c, ind in zip(conseq, idx[:-1]):
            if c is -1:
                new_val = df.loc[idx+1,'MothOut'].values
                df.loc[idx- 1, 'MothOut'] = new_val 
            else:
                new_val = df.loc[idx,'MothOut'].values
                df.loc[idx- 1, 'MothOut'] = new_val 
        if c != -1:
            new_val = df.loc[idx[-1],'MothOut']
            df.loc[idx[-1]- 1, 'MothOut'] = new_val
    else:
        new_val = df.loc[idx,'MothOut'].values
        df.loc[idx- 1, 'MothOut'] = new_val

    df = df.drop(idx, axis = 0)
    df = df.reset_index(drop = True)
    return(df)

In [4]:
direc1 = r"./dataFolders/Output/Step2_DetectMothINOUT/"
filelist = glob.glob(direc1 + '*MothInOut.csv')

In [6]:
direc2 = r"G:\My Drive\Tom-Tanvi\Shared With Bing,Tom and Tanvi\Video Analysis\DeepLabCut-ImageAnalysis\take5\outputFromDLC\VideoResults\EntireDataSet"
file_dlc = glob.glob(direc2 + r"\*.h5")

In [13]:
new_path = r"./dataFolders/Output/Step3_TrialLabelling_v2/"

try:
    os.makedirs(new_path)
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

In [21]:
num_trial = pd.DataFrame(data = None, columns = ['name', 'initial_num', 'dlc_filter', 'total_len_filter', 'final_num'], dtype = int)
num_trial.head()

,name,initial_num,dlc_filter,total_len_filter,final_num


In [25]:
for i, file in enumerate(filelist):
    df = pd.read_csv(file)
    total_trial = len(df)
    name = os.path.basename(file)[:-14]
    
    # Get the stats on DLCTracking
    file_matched = [f for f in file_dlc if os.path.basename(file)[:-14] in f]
    temp = pd.read_hdf(file_matched[0])
    
    DLC = temp.loc[slice(None),(slice(None),'proboscisTip')].copy()
    DLC.columns = DLC.columns.droplevel(level = [0,1])

    mean_col = []
    
    for row in df.iterrows():
        data = row[1]

        In = data.MothIN
        out = data.MothOut

        mu = DLC.loc[In:out, 'likelihood'].mean(skipna = True)
        mean_col.append(mu)
    df.loc[:,'mean'] = mean_col

    #drop trials based  on length and prob(proboscis)
    filtered_df = df[df['mean'] > m].copy()
    mean_filt_trials = len(filtered_df)
    
    filt_df = filtered_df[filtered_df.FramesInView > visit_length]
    filt_df = filt_df.reset_index(drop = True)
    dwell_time_trials = len(filt_df)
    
    #merge trials
    frames_inbetween =  filt_df.MothIN.values[1:] - filt_df.MothOut.values[:-1]
    frames_inbetween = frames_inbetween.astype(np.float64)
    frames_inbetween = np.insert(frames_inbetween, 0, np.nan, axis = 0)
    filt_df['frames_inbetween'] = frames_inbetween
    
    final_df = MergeVisits(filt_df, frames_btw_visits)
    final_trials = len(final_df)
    
    final_df = final_df.drop(columns = ['mean', 'frames_inbetween', 'FramesInView'])
    final_df.to_csv(new_path + name + '_trueTrials.csv')
    
    num_trial.loc[i, 'name'] = name
    num_trial.loc[i,'initial_num'] = total_trial
    num_trial.loc[i,'dlc_filter'] = mean_filt_trials
    num_trial.loc[i,'total_len_filter'] = dwell_time_trials
    num_trial.loc[i,'final_num'] = final_trials
num_trial.to_csv(new_path + 'trialSelectionStats.csv')